# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [25]:
from azureml.core import Workspace, Experiment
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.core.model import Model
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice, loguniform
from azureml.core.compute import ComputeTarget, AmlCompute
import sklearn
import os
import joblib

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [26]:
ws = Workspace.from_config()
experiment_name = 'hyperdrive'

experiment=Experiment(ws, experiment_name)

## Start compute cluster

In [5]:
cpu_cluster_name = "ML-Cluster"
compute_config = AmlCompute.provisioning_configuration("Standard_D2_V2", min_nodes=0, max_nodes=2)
cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
cpu_cluster.wait_for_completion(show_output=True)

InProgress....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [30]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.05, evaluation_interval=5, delay_evaluation=10)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    "--C": loguniform(-10, 10),
    "--max_iter": choice(100,200,400)
})

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn("./", compute_target=cpu_cluster, entry_script="train.py")

hyperdrive_run_config = HyperDriveConfig(hyperparameter_sampling=param_sampling, 
                                    estimator=estimator, 
                                    policy=early_termination_policy,
                                    primary_metric_name="Accuracy", 
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,  
                                    max_total_runs=8)

In [31]:
#TODO: Submit your experiment
hdr = experiment.submit(hyperdrive_run_config, show_output=True)
RunDetails(hdr).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [33]:
best_run = hdr.get_best_run_by_primary_metric()
best_run_metric = best_run.get_metrics()

print(best_run)
print(best_run_metric)

best_run.get_file_names()

Run(Experiment: hyperdrive,
Id: HD_a53ab7a9-850d-4007-a7b3-5f382e4e6753_4,
Type: azureml.scriptrun,
Status: Completed)
{'Regularization Strength:': 1.9036038700272717, 'Max iterations:': 200, 'Accuracy': 0.9125960061443933}


['azureml-logs/55_azureml-execution-tvmps_352e69e9d8d6b50af6e27061fef4f34eb0d389aee6c109706fb3a0d92e2109ea_d.txt',
 'azureml-logs/65_job_prep-tvmps_352e69e9d8d6b50af6e27061fef4f34eb0d389aee6c109706fb3a0d92e2109ea_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_352e69e9d8d6b50af6e27061fef4f34eb0d389aee6c109706fb3a0d92e2109ea_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'logs/azureml/105_azureml.log',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log',
 'outputs/model.joblib']

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

